I am going to calculate the level-average absorption Oscillator Strength value for transation n1 to n2, for n2 > n1

Using Gordon & Sorochenko (2002) notation in this formula, but in the code, n2 correspond to the lower state as in D. Hoang paper

\begin{equation}
f(n_1,n_2)n_1^2 =  \sum_{l_1=1}^{n_1-1} [(2l_1 +1) f(n_1,l_1;n_2,l_1 -1)] + \sum_{l_1 =0}^{n_1 -1} [(2l_1 +1) f(n_1,l_1;n_2,l_2)]
\end{equation}

I calculate the radiative data as calculated in my other Notebook radiative_data_for_transition_111_110_v1_2, where the value f(n1,l1;n2,l2) is calculated using the formula in D. Hoang paper:

\begin{equation}
f(n',l';n,l) = \frac{1}{3} \quad (\frac{\nu}{\mathrm{Ry}}) \quad [ \frac{max(\ell_1,\ell_2)}{(2\ell_2 + 1)} ] \quad [ R(n_1,\ell_1;n_2,\ell_2) ]^2
\end{equation}

This formula should be the same than in Gordon & Sorochengo (2002) Page 287:

\begin{equation}
f(n',l';n,l) = \frac{1}{2l_1 +1 } \sum_{m_1=-l_1}^{l1} \sum_{m_2=-l_2}^{l_2} f(n_1,l_1,m_1;n_2,l_2,m_2) = \frac{1}{3}(\frac{1}{n_1^2} - \frac{1}{n_2^2}) \frac{max(l_1,l_2)}{2l_1+1} \frac{1}{ap_0^2} * [R(n_2,l_2;n_1,l_1)]^2
\end{equation}

where
\begin{equation}
ap_0 \equiv ( \frac{R_{\infty}}{R_M} ) \frac{a_0}{Z}
\end{equation}

In [1]:
import numpy as np
import scipy.constants

from arc import *                 #Import ARC (Alkali Rydberg Calculator)

# Working with a Hydrogen atom
atom = Hydrogen()

# Define the transition we are studing
# Upper state
n1 = 111
# Lower state
n2 = 110

# Transition frequency: used in second and third calculation
# It is the same for n1 and n2.

# ARC
tfreq = abs(atom.getTransitionFrequency(n1,0,0.5,n2,1,0.5))

# I am going to use a NumPy array to store the calculations data.
# 2 Dimensions: Rows are the number of l values calculating
#               Columns 9, defined as:
#                   0 : l value use in all calculations in this row
#                   1 : R(n1,l;n2,l-1) - using ARC
#                   2 : R(n1,l;n2,l+1) - using ARC
#                   3 : f(n2,l-1;n1,l) - this calculation will use value stored in [1]
#                   4 : f(n2,l+1;n1,l) - this calculation will use value stored in [2]
#                   5 : A(n1,l;n2,l-1) - using D. Hoang-Binh formula and value in [1]
#                   6 : A(n1,l;n2,l+1) - using D. Hoang-Binh formula and value in [2]
#                   7 : A(n1,l;n2,l-1) - using ARC and value in [1]
#                   8 : A(n1,l;n2,l+1) - using ARC and value in [2]
# Define varialbes for column index, for clarity
value_l = 0
R_l_minus1 = 1
R_l_plus1 = 2
f_l_minus1 = 3
f_l_plus1 = 4
A_l_minus1 = 5
A_l_plus1 = 6
A_arc_l_minus1 = 7
A_arc_l_plus1 = 8

# Define the values of l that we are going to calculate: comment the line not used
# Using a Numpy ndarray
# States calculated in the paper
#l_states = [0,1,2,3,4,5,10,20,30,40,50,60,70,80,90,100,105,106,107,108,109,110]
# All possible states
l_states = range(n1)
# All posible states for l, from 0 to n2
#l_states = range(n2)

# Initialize the np array with zeros for the calculations and the type for the resusts
data = np.zeros((len(l_states),8),dtype=np.float64)
# Append the list with l_states, they are cast to float
data = np.column_stack((l_states,data))


#*****************************************************************************************************
# FUNCTION DEFINITIONS
#***************************************************************/

# Function to get the Dipole Moment from ARC and check that l state is valid
def getDipoleMoment( n1, l1, j1, n2, l2, j2 ):
    
    if ((l1 >= n1) or (l2 >= n2) or (l1 < 0) or (l2 < 0)):
        return float('nan')

    # Transition frequency Hz
    if ( ( (j1 == (l1+0.5)) or (j1 == (l1-0.5))) and ((j2 == (l2+0.5)) or (j2 ==(l2-0.5) ) ) )  :
        # Eq 2 in 
        R = atom.getRadialMatrixElement(n1, l1, j1, n2, l2, j2)
        return R
    else:
        return float('nan')
    
# Defining a function to calculate AOS
def getAbsorptionOscillatorStrength( n1, l1, j1, n2, l2, j2, RadialInt, tfreq ):
    #"This function calculates the absorption oscillator strength for the Radial Integral provided"
    Rydberg_Hz = scipy.constants.value(u'Rydberg constant times c in Hz')
    
    if RadialInt == 'nan':
        return float('nan')
    
    if ((l1 >= n1) or (l2 >= n2) or (l1 < 0) or (l2 < 0)):
        return float('nan')

    # Transition frequency Hz
    if ( ( (j1 == (l1+0.5)) or (j1 == (l1-0.5))) and ((j2 == (l2+0.5)) or (j2 ==(l2-0.5) ) ) )  :
     #   tfreq = atom.getTransitionFrequency(n1,l1,j1,n2,l2,j2)

        # Eq 2 in 
        f_n2_n1 = 1/3 * (tfreq/Rydberg_Hz) * (max(l1,l2)/(2*l2 +1)) * RadialInt
        return abs(f_n2_n1)
    else:
        return float('nan')
    
# Calculate R first (dipole moment) that we need to calculate the Oscillator Strenth
    
i=0 # Iterator for the row we are working on
for l in data[:,value_l]:
    # Each l is the value of l for this calculation
    l_int = int(l)
    # Calculating R
    # Here I square the dipole moment, so it is saved squared
    data[i][R_l_minus1] = getDipoleMoment(n1, l_int, l+0.5, n2, l_int-1, l-0.5)**2
    data[i][R_l_plus1] =  getDipoleMoment(n1, l_int, l+0.5, n2, l_int+1, l+0.5)**2
    # Calculating f
    data[i][f_l_minus1] = getAbsorptionOscillatorStrength(n1, l_int, l_int+0.5, n2, l_int-1, l_int-0.5,data[i][R_l_minus1],tfreq)
    data[i][f_l_plus1] = getAbsorptionOscillatorStrength(n1, l_int, l_int+0.5, n2, l_int+1, l_int+0.5,data[i][R_l_plus1],tfreq)

    # Printing the data to make sure it is correct
    #print('R%3d\t%9.3e\t\t%.3e ' % (l,data[i][R_l_minus1],data[i][R_l_plus1]))
    #print('f%3d\t%9.3e\t\t%.3e ' % (l,data[i][f_l_minus1],data[i][f_l_plus1]))
    
    i+=1
    

In [2]:
# Now I am going to calculate the Oscillator Strength for transation n2 to n1
# States numbers are different from the original formula in Gordon & Sorochenko

# First Sum
totalSum1 = 0
for l_1m in range(1,n2-1):
    totalSum1 += (2*l_1m+1)*data[l_1m][f_l_minus1]

totalSum2 = 0
for l_1p in range(0,n2-1):
    totalSum2 += (2*l_1p+1)*data[l_1p][f_l_plus1]
    
# f(n2,n1) is the sum of both divided by n2 square (I changed the sub)
fn2_n1 = (totalSum1 + totalSum2)/n2**2


print('The Oscillator Strength for transition %d to %d is %f' % (n1,n2,fn2_n1))

The Oscillator Strength for transition 111 to 110 is 20.221326


Results:

* The Oscillator Strength for transition 101 to 100 is 18.315134
* The Oscillator Strength for transition 111 to 110 is 20.221326